In [46]:
from aij_multiagent_rl.env import AijMultiagentEnv
import yaml
import torch
from vg_lib.net.policy import Actor, Critic
from vg_lib.env.vec_env import create_vec_env

In [47]:
env = create_vec_env(AijMultiagentEnv, 2)

In [48]:
obs = env.reset()
world = env.state()

In [57]:
res = env.step([{f'agent_{i}':1 for i in range(8)} for _ in range(2)])

In [60]:
res

([{'agent_0': {'image': array([[[  0,   0,   0],
            [  0,   0,   0],
            [  0,   0,   0],
            ...,
            [  0,   0,   0],
            [  0,   0,   0],
            [  0,   0,   0]],
    
           [[  0,   0,   0],
            [  0,   0,   0],
            [  0,   0,   0],
            ...,
            [  0,   0,   0],
            [  0,   0,   0],
            [  0,   0,   0]],
    
           [[  0,   0,   0],
            [  0,   0,   0],
            [  0,   0,   0],
            ...,
            [  0,   0,   0],
            [  0,   0,   0],
            [  0,   0,   0]],
    
           ...,
    
           [[  6, 133,   6],
            [  5, 132,   5],
            [ 10, 137,  10],
            ...,
            [  4, 131,   4],
            [  0, 123,   0],
            [  9, 136,   9]],
    
           [[ 15, 142,  15],
            [  0, 116,   0],
            [  0, 114,   0],
            ...,
            [  0, 110,   0],
            [ 12, 139,  12],
         

In [67]:
env.state()[0]

{'image': array([[[ 12, 139,  12],
         [  6, 131,   6],
         [  7, 132,   7],
         ...,
         [  5, 124,   5],
         [  5, 125,   5],
         [  6, 128,   6]],
 
        [[ 14, 141,  14],
         [  7, 133,   7],
         [  5, 127,   5],
         ...,
         [  9, 136,   9],
         [ 12, 136,  12],
         [  3, 127,   3]],
 
        [[  7, 132,   7],
         [  4, 126,   4],
         [  4, 130,   4],
         ...,
         [  6, 129,   6],
         [  8, 133,   8],
         [  7, 130,   7]],
 
        ...,
 
        [[  5, 128,   5],
         [  1, 120,   1],
         [  8, 133,   8],
         ...,
         [  9, 134,   9],
         [  7, 128,   7],
         [  4, 124,   4]],
 
        [[  1, 123,   1],
         [  3, 129,   3],
         [  3, 123,   3],
         ...,
         [  3, 120,   3],
         [  4, 127,   4],
         [  8, 132,   8]],
 
        [[  4, 125,   4],
         [  2, 125,   2],
         [  9, 132,   9],
         ...,
         [  3, 126,

In [6]:
one_env = AijMultiagentEnv()

In [12]:
h,w,c = one_env.observation_space('agent_0')['image'].shape

(60, 60, 3)

In [31]:
initial_world_state = env.reset()

In [ ]:
from tra

In [39]:
initial_world_state[0][1]

{'agent_0': {'ecology_score': 100,
  'num_trash': 0,
  'num_resource': 6,
  'dead_ecology': False},
 'agent_1': {'ecology_score': 100,
  'num_trash': 0,
  'num_resource': 5,
  'dead_ecology': False},
 'agent_2': {'ecology_score': 100,
  'num_trash': 0,
  'num_resource': 5,
  'dead_ecology': False},
 'agent_3': {'ecology_score': 100,
  'num_trash': 0,
  'num_resource': 6,
  'dead_ecology': False},
 'agent_4': {'ecology_score': 100,
  'num_trash': 0,
  'num_resource': 7,
  'dead_ecology': False},
 'agent_5': {'ecology_score': 100,
  'num_trash': 0,
  'num_resource': 9,
  'dead_ecology': False},
 'agent_6': {'ecology_score': 100,
  'num_trash': 0,
  'num_resource': 4,
  'dead_ecology': False},
 'agent_7': {'ecology_score': 100,
  'num_trash': 0,
  'num_resource': 4,
  'dead_ecology': False}}

In [13]:
n_actions = one_env.action_space('agent_0')

In [ ]:
agent = Agent(obs)

In [24]:
one_env.state()['image'].shape

(110, 110, 3)

Process Process-2:
Process Process-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/vladeck/aij_multiagent_solution_vgteam/vg_lib/env/vec_env.py", line 21, in worker_process
    out = env.step(actions)
          ^^^^^^^^^^^^^^^^^
  File "/Users/vladeck/aij_multiagent_solution_vgteam/aij_multiagent_rl/aij_multiagent_rl/env.py", line 445, in step
    self._make_action(agent_id=agent, action_id=actions[agent])
                                                ~~~~~~~^^^^^^^
IndexError: invalid index to scalar variable.
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    se

KeyboardInterrupt: 

In [41]:
import numpy as np